In [ ]:
!ls *.sh

In [32]:
# Previously for CCSM4 model, which does not have max/min relative humidity needed to find vapor pressure deficit
#with open('macav2metdata-allnetcdf_CONUS.sh') as fo:
#    lines = fo.readlines()

with open('MIROC5_historical_future_cloudVegET.sh') as fo:
    lines = fo.readlines() 

'wget -nc -c -nd "http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_pr_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc" -O macav2metdata_pr_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc  \n'

In [33]:
# explore some of the output
print(lines)

words = lines[1].split()
for w in words:
    if 'http' in w:
        print(w)
        http_line = w
print(http_line)

http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
print(http_root)

wget -nc -c -nd "http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_pr_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc" -O macav2metdata_pr_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc  

"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_pr_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc"
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_pr_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc"
http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_pr_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc


In [1]:
import pandas as pd  
import os

with open('MIROC5_historical_future_cloudVegET.sh') as fo:
    lines = fo.readlines()

# Extract name of netCDF from shell script
outs=[]
for line in lines:
    if '-O' in line:
        out=line.split('-O')[1][1:]
        out = out.rstrip()
        out = out.split('.nc')[0]
        print(out)
        outs.append(out)

# break down the name of each file into each metadata component
dfis=[]
for out in outs:
    a=out.split('_')
    dfi = {'dataset':a[0], 'measure':a[1], 'model':a[2], 'r6':a[3], 'epoch':a[4], 'start_year':a[5], 'end_year':a[6],
          'region':a[7], 'frequency':a[8]}
    dfis.append(dfi)

# Create a dataframe consisting of metdata info on each of the netCDFs 
df = pd.DataFrame(dfis)

# Get climate variable names - will be needed later when moving into the cloud
# print(df['measure'].values)

home = os.getenv('HOME')
wget = 'wget -nc -c -nd ' 

print(df)

macav2metdata_pr_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_historical_1955_1959_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_historical_1960_1964_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_historical_1970_1974_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_historical_1975_1979_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_historical_1980_1984_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_historical_1985_1989_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_historical_1990_1994_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_historical_1995_1999_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_historical_2000_2004_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_historical_2005_2005_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_rcp45_2011_2015_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_rcp45_2016_2020_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_rcp45_2021_2025_CONUS_da

In [2]:
import boto3
def s3_push_delete_local(local_file, bucket, bucket_filepath):
        """
        This function will move the model outputs from a local folder to a cloud bucket.
        :param local_file: path the the local geo file
        :param outpath: path of a directory to be created in the cloud bucket
        :param bucket: name of the cloud bucket = 'dev-et-data'
        :param bucket_folder: "folder" in cloud bucket  = 'v1DRB_outputs'
        :return:
        """

        s3 = boto3.client('s3')
        with open(local_file, "rb") as f:
            print(bucket, bucket_filepath)
            s3.upload_fileobj(f, bucket, bucket_filepath)
        os.remove(local_file)


In [38]:
pr_df = df[df['measure'] == 'pr'] # extract all of the rows that have 'pr' or precipitation within the netcdf file name
pr_lines = lines[1:32] # 1 to 31 are the rows associated with precipitation netCDF files

for i in range(0,len(pr_df.index)):
    words = pr_lines[i].split()
    for w in words:
        if 'http' in w:
            print(w)
            http_line = w

    http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    
    myd = dict(pr_df.loc[i])
    
    to_dir= home + '/junkbox/' + myd['measure']
    
    try:
        os.mkdir(to_dir);
    except:
        pass
    
    myl = list(pr_df.loc[i]) # get each precipitation row 
    
    crafted_file_name = '_'.join(myl) + '.nc'
    
    wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    
    wget_cmd2 = wget + wget_cmd
    
    os.system(wget_cmd2)
    
    bucket = 'dev-et-data'
    
    local_file = to_dir + '/' + crafted_file_name
    
    # output filepath moving from the mini-Pangeo to the Amazon s3 databucket
    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    s3_push_delete_local(local_file, bucket, bucket_filepath)

# Everything below this point is the same code, except for different variables, which requires different indexing values
    

"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_pr_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc"
dev-et-data raw/pr/macav2metdata_pr_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_pr_MIROC5_r1i1p1_historical_1955_1959_CONUS_daily.nc"
dev-et-data raw/pr/macav2metdata_pr_MIROC5_r1i1p1_historical_1955_1959_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_pr_MIROC5_r1i1p1_historical_1960_1964_CONUS_daily.nc"
dev-et-data raw/pr/macav2metdata_pr_MIROC5_r1i1p1_historical_1960_1964_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_pr_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc"
dev-et-data raw/pr/macav2metdata_pr_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5

In [35]:
# rsds_df = df[df['measure'] == 'rsds']
# print(rsds_df)
# rsds_lines = lines[32:63]
# print(rsds_lines)
# len(rsds_df.index)
# rsds_df.loc[31]

rsds_df = df[df['measure'] == 'rsds']
rsds_lines = lines[94:125]

for i in range(0,len(rsds_df.index)):
    words = rsds_lines[i].split()
    for w in words:
        if 'http' in w:
            print(w)
            http_line = w

    http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    
    myd = dict(rsds_df.loc[i+93]) # +31 is done to identify the row numbers from the beginning of the dataframe (i.e. doesn't start at 0)
    
    to_dir= home + '/junkbox/' + myd['measure']
    
    try:
        os.mkdir(to_dir);
    except:
        pass
    
    myl = list(rsds_df.loc[i+93])
    
    crafted_file_name = '_'.join(myl) + '.nc'
    
    wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    
    wget_cmd2 = wget + wget_cmd
    
    os.system(wget_cmd2)
    
    bucket = 'dev-et-data'

    local_file = to_dir + '/' + crafted_file_name

    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    s3_push_delete_local(local_file, bucket, bucket_filepath)

"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_rsds_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc"
dev-et-data raw/rsds/macav2metdata_rsds_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_rsds_MIROC5_r1i1p1_historical_1955_1959_CONUS_daily.nc"
dev-et-data raw/rsds/macav2metdata_rsds_MIROC5_r1i1p1_historical_1955_1959_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_rsds_MIROC5_r1i1p1_historical_1960_1964_CONUS_daily.nc"
dev-et-data raw/rsds/macav2metdata_rsds_MIROC5_r1i1p1_historical_1960_1964_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_rsds_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc"
dev-et-data raw/rsds/macav2metdata_rsds_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/

In [ ]:
tasmin_df = df[df['measure'] == 'tasmin']
tasmin_lines = lines[125:156]

for i in range(0,len(tasmin_df.index)):
    words = tasmin_lines[i].split()
    for w in words:
        if 'http' in w:
            print(w)
            http_line = w

    http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    
    myd = dict(tasmin_df.loc[i+124])
    
    to_dir= home + '/junkbox/' + myd['measure']
    
    try:
        os.mkdir(to_dir);
    except:
        pass
    
    myl = list(tasmin_df.loc[i+124])
    
    crafted_file_name = '_'.join(myl) + '.nc'
    
    wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    
    wget_cmd2 = wget + wget_cmd
    
    os.system(wget_cmd2)
    
    bucket = 'dev-et-data'

    local_file = to_dir + '/' + crafted_file_name

    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    s3_push_delete_local(local_file, bucket, bucket_filepath)

"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_tasmin_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc"
dev-et-data raw/tasmin/macav2metdata_tasmin_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_tasmin_MIROC5_r1i1p1_historical_1955_1959_CONUS_daily.nc"
dev-et-data raw/tasmin/macav2metdata_tasmin_MIROC5_r1i1p1_historical_1955_1959_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_tasmin_MIROC5_r1i1p1_historical_1960_1964_CONUS_daily.nc"
dev-et-data raw/tasmin/macav2metdata_tasmin_MIROC5_r1i1p1_historical_1960_1964_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_tasmin_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc"
dev-et-data raw/tasmin/macav2metdata_tasmin_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc
"http://thredds.northwestkno

In [ ]:
tasmax_df = df[df['measure'] == 'tasmax']
tasmax_lines = lines[156:187]

for i in range(0,len(tasmax_df.index)):
    words = tasmax_lines[i].split()
    for w in words:
        if 'http' in w:
            print(w)
            http_line = w

    http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    
    myd = dict(tasmax_df.loc[i+155])
    
    to_dir= home + '/junkbox/' + myd['measure']
    
    try:
        os.mkdir(to_dir);
    except:
        pass
    
    myl = list(tasmax_df.loc[i+155])
    
    crafted_file_name = '_'.join(myl) + '.nc'
    
    wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    
    wget_cmd2 = wget + wget_cmd
    
    os.system(wget_cmd2)
    
    bucket = 'dev-et-data'

    local_file = to_dir + '/' + crafted_file_name

    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    s3_push_delete_local(local_file, bucket, bucket_filepath)

In [21]:
uas_df = df[df['measure'] == 'uas']
uas_lines = lines[187:218]

for i in range(0,len(uas_df.index)):
    words = uas_lines[i].split()
    for w in words:
        if 'http' in w:
            print(w)
            http_line = w

    http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    
    myd = dict(uas_df.loc[i+186])
    
    to_dir= home + '/junkbox/' + myd['measure']
    
    try:
        os.mkdir(to_dir);
    except:
        pass
    
    myl = list(uas_df.loc[i+186])
    
    crafted_file_name = '_'.join(myl) + '.nc'
    
    wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    
    wget_cmd2 = wget + wget_cmd
    
    os.system(wget_cmd2)
    
    bucket = 'dev-et-data'

    local_file = to_dir + '/' + crafted_file_name

    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    s3_push_delete_local(local_file, bucket, bucket_filepath)

"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_uas_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc"
dev-et-data raw/uas/macav2metdata_uas_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_uas_MIROC5_r1i1p1_historical_1955_1959_CONUS_daily.nc"
dev-et-data raw/uas/macav2metdata_uas_MIROC5_r1i1p1_historical_1955_1959_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_uas_MIROC5_r1i1p1_historical_1960_1964_CONUS_daily.nc"
dev-et-data raw/uas/macav2metdata_uas_MIROC5_r1i1p1_historical_1960_1964_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_uas_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc"
dev-et-data raw/uas/macav2metdata_uas_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/M

In [30]:
vas_df = df[df['measure'] == 'vas']
vas_lines = lines[218:249]

for i in range(0,len(vas_df.index)):
    words = vas_lines[i].split()
    for w in words:
        if 'http' in w:
            print(w)
            http_line = w

    http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    
    myd = dict(vas_df.loc[i+217])
    
    to_dir= home + '/junkbox/' + myd['measure']
    
    try:
        os.mkdir(to_dir);
    except:
        pass
    
    myl = list(vas_df.loc[i+217])
    
    crafted_file_name = '_'.join(myl) + '.nc'
    
    wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    
    wget_cmd2 = wget + wget_cmd
    
    os.system(wget_cmd2)
    
    bucket = 'dev-et-data'

    local_file = to_dir + '/' + crafted_file_name

    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    s3_push_delete_local(local_file, bucket, bucket_filepath)

"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_vas_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc"
dev-et-data raw/vas/macav2metdata_vas_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_vas_MIROC5_r1i1p1_historical_1955_1959_CONUS_daily.nc"
dev-et-data raw/vas/macav2metdata_vas_MIROC5_r1i1p1_historical_1955_1959_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_vas_MIROC5_r1i1p1_historical_1960_1964_CONUS_daily.nc"
dev-et-data raw/vas/macav2metdata_vas_MIROC5_r1i1p1_historical_1960_1964_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_vas_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc"
dev-et-data raw/vas/macav2metdata_vas_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/M

In [34]:
rhsmax_df = df[df['measure'] == 'rhsmax']
rhsmax_lines = lines[32:63]

for i in range(0,len(rhsmax_df.index)):
    words = rhsmax_lines[i].split()
    for w in words:
        if 'http' in w:
            print(w)
            http_line = w

    http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    
    myd = dict(rhsmax_df.loc[i+31])
    
    to_dir= home + '/junkbox/' + myd['measure']
    
    try:
        os.mkdir(to_dir);
    except:
        pass
    
    myl = list(rhsmax_df.loc[i+31])
    
    crafted_file_name = '_'.join(myl) + '.nc'
    
    wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    
    wget_cmd2 = wget + wget_cmd
    
    os.system(wget_cmd2)
    
    bucket = 'dev-et-data'

    local_file = to_dir + '/' + crafted_file_name

    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    s3_push_delete_local(local_file, bucket, bucket_filepath)

"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_rhsmax_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc"
dev-et-data raw/rhsmax/macav2metdata_rhsmax_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_rhsmax_MIROC5_r1i1p1_historical_1955_1959_CONUS_daily.nc"
dev-et-data raw/rhsmax/macav2metdata_rhsmax_MIROC5_r1i1p1_historical_1955_1959_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_rhsmax_MIROC5_r1i1p1_historical_1960_1964_CONUS_daily.nc"
dev-et-data raw/rhsmax/macav2metdata_rhsmax_MIROC5_r1i1p1_historical_1960_1964_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_rhsmax_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc"
dev-et-data raw/rhsmax/macav2metdata_rhsmax_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc
"http://thredds.northwestkno

In [3]:
rhsmin_df = df[df['measure'] == 'rhsmin']
rhsmin_lines = lines[63:94]

for i in range(0,len(rhsmin_df.index)):
    words = rhsmin_lines[i].split()
    for w in words:
        if 'http' in w:
            print(w)
            http_line = w

    http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    
    myd = dict(rhsmin_df.loc[i+62])
    
    to_dir= home + '/junkbox/' + myd['measure']
    
    try:
        os.mkdir(to_dir);
    except:
        pass
    
    myl = list(rhsmin_df.loc[i+62])
    
    crafted_file_name = '_'.join(myl) + '.nc'
    
    wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    
    wget_cmd2 = wget + wget_cmd
    
    os.system(wget_cmd2)
    
    bucket = 'dev-et-data'

    local_file = to_dir + '/' + crafted_file_name

    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    s3_push_delete_local(local_file, bucket, bucket_filepath)

"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_rhsmin_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc"
dev-et-data raw/rhsmin/macav2metdata_rhsmin_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_rhsmin_MIROC5_r1i1p1_historical_1955_1959_CONUS_daily.nc"
dev-et-data raw/rhsmin/macav2metdata_rhsmin_MIROC5_r1i1p1_historical_1955_1959_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_rhsmin_MIROC5_r1i1p1_historical_1960_1964_CONUS_daily.nc"
dev-et-data raw/rhsmin/macav2metdata_rhsmin_MIROC5_r1i1p1_historical_1960_1964_CONUS_daily.nc
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_rhsmin_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc"
dev-et-data raw/rhsmin/macav2metdata_rhsmin_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc
"http://thredds.northwestkno

# Move it to the cloud 

In [ ]:
import boto3
def s3_push_delete_local(local_file, bucket, bucket_filepath):
        """
        This function will move the model outputs from a local folder to a cloud bucket.
        :param local_file: path the the local geo file
        :param outpath: path of a directory to be created in the cloud bucket
        :param bucket: name of the cloud bucket = 'dev-et-data'
        :param bucket_folder: "folder" in cloud bucket  = 'v1DRB_outputs'
        :return:
        """

        s3 = boto3.client('s3')
        with open(local_file, "rb") as f:
            print(bucket, bucket_filepath)
            s3.upload_fileobj(f, bucket, bucket_filepath)
        os.remove(local_file)


In [ ]:
bucket = 'dev-et-data'

local_file = to_dir + '/' + crafted_file_name

bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name

print(local_file)
print(bucket)
print(bucket_filepath)


In [ ]:
%%time
s3_push_delete_local(local_file, bucket, bucket_filepath)

In [ ]:
len(pr_df.index)

In [ ]:
pr_df = df[df['measure'] == 'pr']

for i in range(0,len(pr_df.index)):
    
    myl = list(pr_df.loc[i])

    crafted_file_name = '_'.join(myl) + '.nc'
    
    bucket = 'dev-et-data'

    local_file = to_dir + '/' + crafted_file_name

    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    s3_push_delete_local(local_file, bucket, bucket_filepath)

In [ ]:
import fsspec


In [ ]:
fs = fsspec.filesystem('s3', anon=False, requester_pays=True)

In [ ]:
file_objects = fs.ls('dev-et-data')
file_objects

In [ ]:
file_objects = fs.ls('dev-et-data/raw/pr/')
file_objects

In [ ]:
next_item = pr_df.loc[pr_df['start_year'] =='1970']

In [ ]:
next_item.values

In [ ]:
myl=list(next_item.values[0])

In [ ]:
myl

In [ ]:
crafted_file_name = '_'.join(myl) + '.nc'
crafted_file_name

In [ ]:
wget_cmd = http_root + '/' + from_dir + '/' + crafted_file_name + ' -O ' + to_dir + '/' + crafted_file_name 
wget_cmd = wget + wget_cmd
wget_cmd


In [ ]:
%%time
os.system(wget_cmd)

In [ ]:
local_file = to_dir + '/' + crafted_file_name
bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
bucket_filepath

In [ ]:
%%time
s3_push_delete_local(local_file, bucket, bucket_filepath)

In [1]:
fs = fsspec.filesystem('s3', anon=False, requester_pays=True)
file_objects = fs.ls('dev-et-data/raw/pr/')
file_objects

NameError: name 'fsspec' is not defined

In [ ]:
!aws s3 ls dev-et-data/raw/pr/